<a href="https://colab.research.google.com/github/syuhraputri27/enviro-edu-chatbot/blob/main/AI_Chatbot_Enviroeducationtools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl sentence_transformers chromadb userdata

ERROR: Could not find a version that satisfies the requirement userdata (from versions: none)
ERROR: No matching distribution found for userdata


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import pandas as pd
import chromadb
import os
import sys
from chromadb.utils import embedding_functions

In [ ]:
# # read dataset
# file_path1 = '/content/data1.xlsx'
# file_path2 = '/content/data2.xlsx'

# df1 = pd.read_excel(file_path1)
# df2 = pd.read_excel(file_path2)

In [ ]:
url_data1 = 'https://docs.google.com/spreadsheets/d/1-l1qJJpSdxjSwAj4AR9Nb0hd7ZV6FCpAtsGEgSvSPLQ/export?format=csv&gid=1254825668'
url_data2 = 'https://docs.google.com/spreadsheets/d/1-l1qJJpSdxjSwAj4AR9Nb0hd7ZV6FCpAtsGEgSvSPLQ/export?format=csv&gid=1906749374'

df_data1 = pd.read_csv(url_data1)
df_data2 = pd.read_csv(url_data2)

In [ ]:
print(df_data1.head())

                       Title  \
0        Air Quality Testers   
1      Water Quality Testers   
2       Soil Quality Testers   
3  Educational Field Studies   
4  Science Labs and Campuses   

                                                Data  \
0  Air quality testers play a vital role in monit...   
1  Water quality testers, including pH meters, tu...   
2  Soil testers such as moisture sensors, pH mete...   
3  Field-based education becomes exponentially mo...   
4  In science labs and campus facilities, environ...   

                                    Augumented Chunk  \
0  Air quality testers play a vital role in monit...   
1  Water quality testers are fundamental tools fo...   
2  Soil testers, such as moisture sensors, pH met...   
3  Field-based education becomes exponentially mo...   
4  In science labs and campus facilities, environ...   

                            URL Page  
0  https://enviroeducationtools.com/  
1  https://enviroeducationtools.com/  
2  https://envir

In [ ]:
print(df_data2.head())

                       Title                   Product Name  \
0  Air Quality & Atmospheric  Portable Air Quality Monitors   
1  Air Quality & Atmospheric          Carbon Dioxide Meters   
2  Air Quality & Atmospheric     Particulate Matter Sensors   
3  Air Quality & Atmospheric               Weather Stations   
4  Air Quality & Atmospheric                Ozone Detectors   

                                                Data  \
0  Our Portable Air Quality Monitors are compact,...   
1  High carbon dioxide levels can negatively impa...   
2  Understanding airborne particulate matter is v...   
3  Our educational weather stations are all-in-on...   
4  Ozone plays a complex role as both a protectiv...   

                                         Detail Desc  \
0  Designed for schools and universities, these p...   
1  Ensure healthy classrooms with accurate CO₂ mo...   
2  Bring science to life with real-time particula...   
3  Hands-on tools to teach students about climate...   
4  E

#Experiment 1

##Prepocessing

In [ ]:
df1, df2 = df_data1, df_data2

**== Data 1 ==**

In [ ]:
df1.tail()

,Title,Data,Augumented Chunk,URL Page
6,Transforming Education with Strategic Environm...,Strategic deployment of environmental testers ...,Strategic deployment of environmental testers ...,https://enviroeducationtools.com/
7,Built for Education Excellence,Our platforms offer educational-grade accuracy...,Our platforms are engineered from the ground u...,https://enviroeducationtools.com/
8,Full Environmental Testing Integration,We deliver an end-to-end solution—combining du...,"We deliver a comprehensive, end-to-end solutio...",https://enviroeducationtools.com/
9,Geographic Reach,We proudly support institutions across the U.S...,We proudly support educational and research in...,https://enviroeducationtools.com/
10,Premium Equipment and Partnerships,Our testers are sourced from globally respecte...,Our commitment to quality begins with our sour...,https://enviroeducationtools.com/


In [ ]:
# cek null

df1.isnull().sum()

,0
Title,0
Data,0
Augumented Chunk,0
URL Page,0


In [ ]:
# cek total kata

df1["word_count"] = df1["Augumented Chunk"].apply(lambda x: len(str(x).split()))

In [ ]:
df1["word_count"]

,word_count
0,310
1,379
2,291
3,273
4,295
5,290
6,291
7,316
8,333
9,243


In [ ]:
def clean_text_df1(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text = re.sub(r'\s+', ' ', text)  # hapus spasi ganda / newline
    return text

**== Data 2 ==**

In [ ]:
df2

,Title,Product Name,Data,Detail Desc,Hardware,Link Hardware Product,Software,Cloud Services,Key Features and Functionalities,Integrations and Compatibility,Benefits for Educators and Students,Applications,Industries We Serve,Relevant U.S. & Canadian Industry Standards,Case Studies,URL Page
0,Air Quality & Atmospheric,Portable Air Quality Monitors,"Our Portable Air Quality Monitors are compact,...","Designed for schools and universities, these p...",High-accuracy gas and particulate sensors (CO₂...,https://gaotek.com/category/environmental/indu...,Intuitive data logging software for trend anal...,Centralized cloud dashboard for device managem...,Real-time monitoring of multiple air quality p...,"Works with Google Sheets, Excel, and most data...",Encourages inquiry-based learning through real...,Classroom demonstrations and experiments\n\nOu...,K-12 Schools \nColleges and Universities \nEnv...,EPA Air Sensor Guidebook \n\nASHRAE 62.1: Vent...,California Public High School (USA)\nA public ...,https://enviroeducationtools.com/air-quality-a...
1,Air Quality & Atmospheric,Carbon Dioxide Meters,High carbon dioxide levels can negatively impa...,Ensure healthy classrooms with accurate CO₂ mo...,High-accuracy CO₂ sensors (NDIR-based): Utiliz...,https://www.gaoetek.com/products/carbon-dioxid...,Intuitive user interface with data logging\n\n...,Remote data access via web portal\n\nReal-time...,Accurate CO₂ level detection in parts per mill...,Compatible with most standard HVAC systems \n\...,Promotes awareness of indoor air quality among...,Classroom air quality monitoring \n\nEnvironme...,NaN,ASHRAE Standard 62.1 \n\nEPA Indoor Air Qualit...,"U.S. – Philadelphia, PA\nA public school distr...",https://enviroeducationtools.com/air-quality-a...
2,Air Quality & Atmospheric,Particulate Matter Sensors,Understanding airborne particulate matter is v...,Bring science to life with real-time particula...,Laser-Scattering Sensor Modules (measuring PM1...,https://gaotek.com/category/environmental/digi...,Real-time data visualization dashboard \n\nHis...,Secure data storage and backup Cloud-based acc...,"Real-time monitoring of PM1.0, PM2.5, and PM10...",Existing weather stations Classroom science la...,Brings real-world environmental science into t...,Middle and high school science curricula\r\nUn...,NaN,U.S. Environmental Protection Agency (EPA) Air...,"Chicago Public High School System (Illinois, U...",https://enviroeducationtools.com/air-quality-a...
3,Air Quality & Atmospheric,Weather Stations,Our educational weather stations are all-in-on...,Hands-on tools to teach students about climate...,All-in-one sensor suite with solar-powered dat...,https://gaotek.com/category/environmental/othe...,Easy-to-use data visualization dashboards\r\nC...,"Secure, real-time data access via desktop and ...","Real-time atmospheric monitoring (temp, pressu...",Compatible with educational platforms such as ...,Promotes environmental literacy and scientific...,K-12 science and geography classrooms\r\nUnive...,NaN,ASTM D1354\r\nNSTA (National Science Teaching ...,"U.S. – Denver Public Schools, Colorado\r\nA di...",https://enviroeducationtools.com/air-quality-a...
4,Air Quality & Atmospheric,Ozone Detectors,Ozone plays a complex role as both a protectiv...,Enable students to explore the science of air ...,Precision electrochemical or UV-based ozone se...,https://www.gaoatek.com/collections/ozone-gas-...,User-friendly desktop dashboard for data loggi...,Optional cloud syncing for remote data access\...,Real-time ambient ozone detection (ppb/ppm)\r\...,Seamlessly integrates with learning platforms ...,Supports STEM curriculum with real-world exper...,High school chemistry and biology labs\r\nColl...,NaN,EPA Air Quality Index (AQI) Guidelines\r\nOSHA...,U.S. (California High School Environmental Lab...,https://enviroeducationtools.com/air-quality-a...
5,Air Quality & Atmospheric,Carbon Monoxide Detectors,"Carbon monoxide is an invisible, odorless, and...","Designed for cla

In [ ]:
# cek null

df2.isnull().sum()

,0
Title,0
Product Name,0
Data,0
Detail Desc,0
Hardware,0
Link Hardware Product,0
Software,0
Cloud Services,0
Key Features and Functionalities,0
Integrations and Compatibility,0


In [ ]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Ganti newline jadi "; "
    text = re.sub(r'\n+', '; ', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text)
    # Hapus titik koma berulang (misal ; ; ;)
    text = re.sub(r'(;[ ;]*)+', '; ', text)
    # Rapikan kalau di awal/akhir ada ; tersisa
    text = text.strip("; ").strip()
    return text

    # text = str(text).strip()
    # text = re.sub(r'\n+', '; ', text)  # ubah newline jadi tanda '; '
    # text = re.sub(r'\s+', ' ', text)   # hilangkan spasi ganda
    # return text

In [ ]:
# terapkan cleaning ke semua kolom kecuali URL Page
for col in df2.columns:
    if col != "URL Page":
        df2[col] = df2[col].apply(clean_text)

In [ ]:
df2

,Title,Product Name,Data,Detail Desc,Hardware,Link Hardware Product,Software,Cloud Services,Key Features and Functionalities,Integrations and Compatibility,Benefits for Educators and Students,Applications,Industries We Serve,Relevant U.S. & Canadian Industry Standards,Case Studies,URL Page
0,Air Quality & Atmospheric,Portable Air Quality Monitors,"Our Portable Air Quality Monitors are compact,...","Designed for schools and universities, these p...",High-accuracy gas and particulate sensors (CO₂...,https://gaotek.com/category/environmental/indu...,Intuitive data logging software for trend anal...,Centralized cloud dashboard for device managem...,Real-time monitoring of multiple air quality p...,"Works with Google Sheets, Excel, and most data...",Encourages inquiry-based learning through real...,Classroom demonstrations and experiments; Outd...,K-12 Schools ; Colleges and Universities ; Env...,EPA Air Sensor Guidebook ; ASHRAE 62.1: Ventil...,California Public High School (USA); A public ...,https://enviroeducationtools.com/air-quality-a...
1,Air Quality & Atmospheric,Carbon Dioxide Meters,High carbon dioxide levels can negatively impa...,Ensure healthy classrooms with accurate CO₂ mo...,High-accuracy CO₂ sensors (NDIR-based): Utiliz...,https://www.gaoetek.com/products/carbon-dioxid...,Intuitive user interface with data logging; Co...,Remote data access via web portal; Real-time a...,Accurate CO₂ level detection in parts per mill...,Compatible with most standard HVAC systems ; S...,Promotes awareness of indoor air quality among...,Classroom air quality monitoring ; Environment...,,ASHRAE Standard 62.1 ; EPA Indoor Air Quality ...,"U.S. – Philadelphia, PA; A public school distr...",https://enviroeducationtools.com/air-quality-a...
2,Air Quality & Atmospheric,Particulate Matter Sensors,Understanding airborne particulate matter is v...,Bring science to life with real-time particula...,Laser-Scattering Sensor Modules (measuring PM1...,https://gaotek.com/category/environmental/digi...,Real-time data visualization dashboard ; Histo...,Secure data storage and backup Cloud-based acc...,"Real-time monitoring of PM1.0, PM2.5, and PM10...",Existing weather stations Classroom science la...,Brings real-world environmental science into t...,Middle and high school science curricula ; Uni...,,U.S. Environmental Protection Agency (EPA) Air...,"Chicago Public High School System (Illinois, U...",https://enviroeducationtools.com/air-quality-a...
3,Air Quality & Atmospheric,Weather Stations,Our educational weather stations are all-in-on...,Hands-on tools to teach students about climate...,All-in-one sensor suite with solar-powered dat...,https://gaotek.com/category/environmental/othe...,Easy-to-use data visualization dashboards ; Cu...,"Secure, real-time data access via desktop and ...","Real-time atmospheric monitoring (temp, pressu...",Compatible with educational platforms such as ...,Promotes environmental literacy and scientific...,K-12 science and geography classrooms ; Univer...,,ASTM D1354 ; NSTA (National Science Teaching A...,"U.S. – Denver Public Schools, Colorado ; A dis...",https://enviroeducationtools.com/air-quality-a...
4,Air Quality & Atmospheric,Ozone Detectors,Ozone plays a complex role as both a protectiv...,Enable students to explore the science of air ...,Precision electrochemical or UV-based ozone se...,https://www.gaoatek.com/collections/ozone-gas-...,User-friendly desktop dashboard for data loggi...,Optional cloud syncing for remote data access ...,Real-time ambient ozone detection (ppb/ppm) ; ...,Seamlessly integrates with learning platforms ...,Supports STEM curriculum with real-world exper...,High school chemistry and biology labs ; Colle...,,EPA Air Quality Index (AQI) Guidelines ; OSHA ...,U.S. (California High School Environmental Lab...,https://enviroeducationtools.com/air-quality-a...
5,Air Quality & Atmospheric,Carbon Monoxide Detectors,"Carbon monoxide is an invisible, odorless, and...","Designed for classrooms and 

In [ ]:
# df2.columns = df2.columns.str.strip()

##**Chunking**

**Chunk** = Metadata/knowledge





**== data 1 ==**

In [ ]:
# jadikan augmented chunk jadi text_chunk (diseragamkan)
df1["text_chunk"] = df1["Augumented Chunk"].fillna(df1["Data"]).apply(clean_text_df1)

In [ ]:
# Tambah kolom lain agar seragam

df1["source"] = "Data 1"
df1["product_name"] = df1["Title"]
df1["url"] = df1["URL Page"]
df1["global_id"] = df1.index.map(lambda i: f"data1_{i}")
df1["chunk_id"] = None  # ga di-chunk, jadi None

In [ ]:
df1_chunk = df1[[
    "global_id", "chunk_id", "source", "Title",
    "product_name", "text_chunk", "url"
]].rename(columns={"Title": "title"})

In [ ]:
df1_chunk.head()

,global_id,chunk_id,source,title,product_name,text_chunk,url
0,data1_0,None,Data 1,Air Quality Testers,Air Quality Testers,Air quality testers play a vital role in monit...,https://enviroeducationtools.com/
1,data1_1,None,Data 1,Water Quality Testers,Water Quality Testers,Water quality testers are fundamental tools fo...,https://enviroeducationtools.com/
2,data1_2,None,Data 1,Soil Quality Testers,Soil Quality Testers,"Soil testers, such as moisture sensors, pH met...",https://enviroeducationtools.com/
3,data1_3,None,Data 1,Educational Field Studies,Educational Field Studies,Field-based education becomes exponentially mo...,https://enviroeducationtools.com/
4,data1_4,None,Data 1,Science Labs and Campuses,Science Labs and Campuses,"In science labs and campus facilities, environ...",https://enviroeducationtools.com/


**== data 2 ==**

In [ ]:
# definisikan pengelompokan semantik kolom data2

group_map = {
    "Description": ["Data", "Detail Desc"],
    "Technical": ["Hardware", "Software", "Cloud Services"],
    "Features": ["Key Features and Functionalities", "Integrations and Compatibility"],
    "Benefits": ["Benefits for Educators and Students", "Applications"],
    "Credibility": ["Relevant U.S. & Canadian Industry Standards", "Case Studies"],
    "Industries": ["Industries We Serve"]
}

In [ ]:
# gabungkan kolom per grup (skip yang kosong)

def combine_group(row, columns):
    texts = []
    for col in columns:
        val = row.get(col, "")
        if val and isinstance(val, str) and val.strip():
            texts.append(val)
    return " ".join(texts).strip()

In [ ]:
# buat text_chunk terstruktur untuk setiap baris produk

final_chunks_df2 = []
for idx, row in df2.iterrows():
    combined_parts = []
    combined_parts.append(f"Title: {row['Title']}")
    combined_parts.append(f"Product Name: {row['Product Name']}")

    for group_name, cols in group_map.items():
        combined_text = combine_group(row, cols)
        if combined_text:
            combined_parts.append(f"{group_name}: {combined_text}")

    # gabungkan semua bagian jadi satu teks panjang
    text_chunk = " | ".join(combined_parts)

    final_chunks_df2.append({
        "global_id": idx,
        "source": "Data 2",
        "title": row["Title"],
        "product_name": row["Product Name"],
        "text_chunk": text_chunk,
        "url": row["URL Page"]
    })

In [ ]:
df2_chunk = pd.DataFrame(final_chunks_df2)

In [ ]:
df2_chunk.head()

,global_id,source,title,product_name,text_chunk,url
0,0,Data 2,Air Quality & Atmospheric,Portable Air Quality Monitors,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
1,1,Data 2,Air Quality & Atmospheric,Carbon Dioxide Meters,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
2,2,Data 2,Air Quality & Atmospheric,Particulate Matter Sensors,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
3,3,Data 2,Air Quality & Atmospheric,Weather Stations,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
4,4,Data 2,Air Quality & Atmospheric,Ozone Detectors,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...


In [ ]:
pd.reset_option('display.max_colwidth')

# show all character
# pd.set_option('display.max_colwidth', None)

In [ ]:
df2_chunk['text_chunk'].head(3)

,text_chunk
0,Title: Air Quality & Atmospheric | Product Nam...
1,Title: Air Quality & Atmospheric | Product Nam...
2,Title: Air Quality & Atmospheric | Product Nam...


In [ ]:
# cek total kata

df2_chunk["word_count"] = df2_chunk["text_chunk"].apply(lambda x: len(str(x).split()))

In [ ]:
df2_chunk["word_count"]

,word_count
0,481
1,602
2,611
3,588
4,617
5,814
6,581
7,568
8,585
9,660


In [ ]:
# membagi chunk supaya max_words=400, dg overlap=50

def chunk_text(text, max_words=400, overlap=50, min_words=150):
    """Pisahkan teks panjang jadi beberapa chunk agar tidak lewat max_words"""
    if not isinstance(text, str) or not text.strip():
        return []

    words = text.split()
    n = len(words)
    chunks = []

    start = 0
    while start < n:
        end = min(start + max_words, n)
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        if end == n:  # stop kalau sudah akhir
            break
        start = end - overlap  # overlap kata

    # gabungkan chunk terakhir kalau terlalu kecil
    if len(chunks) > 1 and len(chunks[-1].split()) < min_words:
        chunks[-2] += " " + chunks[-1]
        chunks.pop()

    return chunks


In [ ]:
# Iterasi baris satu per satu, simpan hasil ke list
rows = []
for i, row in df2_chunk.iterrows():
    text = row.get("text_chunk", "")
    chunks = chunk_text(text, max_words=400, overlap=50) #default 400, 50

    # langsung skip kalau kosong
    if not chunks:
        continue

    for j, chunk in enumerate(chunks):
        rows.append((
            f"data2_{i}",
            f"data2_{i}_chunk{j}",
            "Data 2",
            row.get("title", ""),
            row.get("product_name", ""),
            chunk,
            row.get("url", ""),
            len(chunk.split())
        ))

In [ ]:
# dataframe hasil chunking

df2_final_chunks = []

for i, row in df2_chunk.iterrows():
    chunks = chunk_text(row["text_chunk"], max_words=400, overlap=50) #biar bisa custom
    for j, chunk in enumerate(chunks):
        df2_final_chunks.append({
            "global_id": f"data2_{i}",
            "chunk_id": f"data2_{i}_chunk{j}",
            "source": "Data 2",
            "title": row["title"],
            "product_name": row["product_name"],
            "text_chunk": chunk,
            "url": row["url"],
            # "word_count": len(chunk.split())
        })

df2_final_chunk = pd.DataFrame(df2_final_chunks)

In [ ]:
print("Total chunk:", len(df2_final_chunk))
# print("Rata-rata panjang kata:", df2_final_chunk["word_count"].mean())
df2_final_chunk.head(3)

Total chunk: 35


,global_id,chunk_id,source,title,product_name,text_chunk,url
0,data2_0,data2_0_chunk0,Data 2,Air Quality & Atmospheric,Portable Air Quality Monitors,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
1,data2_1,data2_1_chunk0,Data 2,Air Quality & Atmospheric,Carbon Dioxide Meters,Title: Air Quality & Atmospheric | Product Nam...,https://enviroeducationtools.com/air-quality-a...
2,data2_1,data2_1_chunk1,Data 2,Air Quality & Atmospheric,Carbon Dioxide Meters,vocational training centers ; Cross-platform a...,https://enviroeducationtools.com/air-quality-a...


In [ ]:
# cek total kata

df2_final_chunk["word_count"] = df2_final_chunk["text_chunk"].apply(lambda x: len(str(x).split()))

df2_final_chunk["word_count"]

,word_count
0,531
1,400
2,252
3,400
4,261
5,400
6,238
7,400
8,267
9,400


In [ ]:
# combine df1_chunk + df2_final_chunk
combined_df = pd.concat([df1_chunk, df2_final_chunk], ignore_index=True)


In [ ]:
combined_df.info()
combined_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   global_id     46 non-null     object 
 1   chunk_id      35 non-null     object 
 2   source        46 non-null     object 
 3   title         46 non-null     object 
 4   product_name  46 non-null     object 
 5   text_chunk    46 non-null     object 
 6   url           46 non-null     object 
 7   word_count    35 non-null     float64
dtypes: float64(1), object(7)
memory usage: 3.0+ KB


,global_id,chunk_id,source,title,product_name,text_chunk,url,word_count
0,data1_0,None,Data 1,Air Quality Testers,Air Quality Testers,Air quality testers play a vital role in monit...,https://enviroeducationtools.com/,NaN
1,data1_1,None,Data 1,Water Quality Testers,Water Quality Testers,Water quality testers are fundamental tools fo...,https://enviroeducationtools.com/,NaN
2,data1_2,None,Data 1,Soil Quality Testers,Soil Quality Testers,"Soil testers, such as moisture sensors, pH met...",https://enviroeducationtools.com/,NaN


# **Chunking Data 2**

In [ ]:
df2.head(5)

df2.tail(5)

,Title,Product Name,Data,Detail Desc,Hardware,Link Hardware Product,Software,Cloud Services,Key Features and Functionalities,Integrations and Compatibility,Benefits for Educators and Students,Applications,Industries We Serve,Relevant U.S. & Canadian Industry Standards,Case Studies,URL Page
13,Soil and Environmental Health,Soil pH Testers,Understanding soil acidity or alkalinity is ke...,Accurate tools to measure soil acidity and alk...,"Durable, waterproof digital soil pH testers: O...",https://www.gaotek.com/colorimeters ; https://...,User-friendly interface for data display and l...,Optional cloud integration for data backup and...,Fast and accurate pH readings from 0 to 14; Au...,Compatible with standard LMS platforms like Go...,Encourages real-world STEM learning through di...,Environmental science labs and coursework; Agr...,,EPA Soil Quality Guidelines; ASTM D4972 – Stan...,United States – Pennsylvania High School STEM ...,https://enviroeducationtools.com/soil-and-envi...
14,Soil and Environmental Health,Light Meters / Lux Meters,Light plays a crucial role in water evaporatio...,Accurate instruments designed to measure light...,Handheld and mountable light sensors with digi...,https://www.gaotek.com/infrared-thermometers-c...,Custom data logging application (Windows/macOS...,Secure cloud-based data storage; Historical da...,Real-time lux and foot-candle measurement; Wid...,Compatible with common environmental data syst...,Empowers educators to integrate real-world env...,Environmental science classes and lab programs...,,ANSI C136.10; NIST Traceable Calibration Stand...,"Jefferson High School, Oregon; Jefferson High’...",https://enviroeducationtools.com/soil-and-envi...
15,Soil and Environmental Health,Sound Level Meters,Though not traditionally associated with hydro...,Helping students understand the impact of soun...,"Compact, handheld digital sound level meters: ...",https://www.gaotek.com/sound-level-meters-cali...,PC connectivity for data download and analysis...,Cloud-based data logging and analysis; User-fr...,Measures sound levels in dBA and dBC weighting...,Compatible with classroom smartboards for visu...,Encourages real-world STEM learning through di...,Classroom experiments on sound propagation and...,,ANSI S1.4; IEC 61672; OSHA Occupational Noise ...,U.S. (California); A public charter high schoo...,https://enviroeducationtools.com/soil-and-envi...
16,Soil and Environmental Health,Radiation Detectors,Monitoring radiation levels in soil and water ...,Equip students and researchers with reliable r...,Portable Geiger-Müller (GM) counter-based dete...,https://www.gaotek.com/radiation-meters ; http...,PC and mobile-compatible applications for data...,Secure cloud-based data logging and storage; H...,"Real-time ionizing radiation detection (alpha,...","Compatible with smart classrooms, interactive ...","Encourages real-world learning in physics, che...",High school and college STEM labs; Environment...,,ANSI N42.33; NRC Title 10 CFR Part 20; IEEE St...,Los Angeles Unified School District (Californi...,https://enviroeducationtools.com/soil-and-envi...
17,Soil and Environmental Health,Temperature and Humidity Data Loggers,Hydrology depends heavily on atmospheric data....,We offer state-of-the-art Temperature and Humi...,High-precision temperature and humidity sensor...,https://www.gaotek.com/hygrometers; https://ww...,Intuitive software for real-time data analysis...,Secure cloud-based data storage and backup; Re...,Real-time Monitoring: Instantly view temperatu...,Compatible with leading environmental software...,,Science Education: Helps students learn about ...,,ASTM E2877-13; ANSI/ASHRAE Standard 55-2020; C...,"University of California, Berkeley; UC Berkele...",https://enviroeducationtools.com/soil-and-envi...


In [ ]:
import pandas as pd
import re # Used to break down case studies

# -------------------------------------------------------------------
# NEW FUNCTION FOR PROCESSING DATA 1 (GENERAL AUGMENTED CHUNK)
# -------------------------------------------------------------------
def process_data_1(df_simple):
    """
    Convert the DataFrame ‘Data 1’ (Title, Data, Augmented Chunk, Page URL)
    to a standard 5-column format.
    """
    all_chunks = []

    for index, row in df_simple.iterrows():
        # In Data 1, ‘Title’ is the main topic.
        main_topic = str(row.get('Title', ''))
        src_url = str(row.get('URL Page', ''))

        # 'Augumented Chunk' is the main text
        chunk_text = str(row.get('Augumented Chunk', ''))

        # Create a chunk title
        chunk_title = f"{main_topic} (General Overview)"

        # Create uique ID
        auto_prefix = "".join([s[0] for s in main_topic.split()]).lower()
        chunk_id = f"{auto_prefix}-overview" # for example: "aqt-overview"

        if chunk_text and chunk_text.strip():
             all_chunks.append({
                'chunk_id': chunk_id,
                'main_topic': main_topic,
                'chunk_title': chunk_title,
                'chunk_text': chunk_text,
                'src_url': src_url
            })

    return pd.DataFrame(all_chunks)

# -------------------------------------------------------------------
# FUNCTION FOR PROCESSING DATA 2 (PRODUCT-SPECIFIC)
# (The name was changed to process_data_2)
# -------------------------------------------------------------------
def process_data_2(df_raw):
    """
    Converting a “wide” DataFrame (df2) into a “long” (chunked) DataFrame
    based on a formula (Hardware, Software, Case Studies, etc.)
    """
    all_chunks = []

    for index, row in df_raw.iterrows():
        main_topic = str(row.get('Product Name', ''))
        src_url = str(row.get('URL Page', ''))
        auto_prefix = "".join([s[0] for s in main_topic.split()]).lower()
        chunk_counter = 1

        def add_chunk(title_template, text, counter):
            if isinstance(text, float): text = ""
            if text and text.strip():
                all_chunks.append({
                    'chunk_id': f"{auto_prefix}-{counter}",
                    'main_topic': main_topic,
                    'chunk_title': title_template.format(main_topic=main_topic),
                    'chunk_text': text.strip(),
                    'src_url': src_url
                })
                return counter + 1
            return counter

        # Formula 1: (Combine Title + Product Name + Data + Detailed Description)
        text = " ".join([
            str(row.get('Title', '')),
            str(row.get('Product Name', '')),
            str(row.get('Data', '')),
            str(row.get('Detail Desc', ''))
        ])
        chunk_counter = add_chunk("{main_topic} (General Description)", text, chunk_counter)

        # Formula 2: (Combine Hardware + Link Hardware Product)
        text = " ".join([
            str(row.get('Hardware', '')),
            str(row.get('Link Hardware Product', ''))
        ])
        chunk_counter = add_chunk("Hardware for {main_topic}", text, chunk_counter)

        # Formula 3: (Combine Software + Cloud Services)
        text = " ".join([
            str(row.get('Software', '')),
            str(row.get('Cloud Services', ''))
        ])
        chunk_counter = add_chunk("Software and Cloud Services for {main_topic}", text, chunk_counter)

        # Formula 4-8: (Copy from specific columns)
        simple_chunks_map = {
            'Key Features and Functionalities': "Key Features of {main_topic}",
            'Integrations and Compatibility': "Integrations for {main_topic}",
            'Benefits for Educators and Students': "Benefits for Educators & Students ({main_topic})",
            'Applications': "Applications for {main_topic}",
            'Relevant U.S. & Canadian Industry Standards': "Industry Standards ({main_topic})"
        }
        for col_name, title_template in simple_chunks_map.items():
            text = str(row.get(col_name, ''))
            chunk_counter = add_chunk(title_template, text, chunk_counter)

        # Formula 9-11: (Copy Case Studies separately)
        case_studies_text = str(row.get('Case Studies', '')).strip()
        if case_studies_text and not pd.isna(case_studies_text):
            case_studies_list = re.split(
                r'[\n\s]+(?=(?:California Public|STEM Charter Academy|University of British Columbia|U\.S\. –|Canada –))',
                case_studies_text
            )
            for cs in case_studies_list:
                cs_text = cs.strip()
                if cs_text:
                    cs_title = " ".join(cs_text.split()[:5]) + "..."
                    chunk_counter = add_chunk(f"Case Study: {cs_title}", cs_text, chunk_counter)
    return pd.DataFrame(all_chunks)

# --- MAIN SCRIPT ---
if __name__ == "__main__":

    # 1. Specify URL
    url_data1 = 'https://docs.google.com/spreadsheets/d/1-l1qJJpSdxjSwAj4AR9Nb0hd7ZV6FCpAtsGEgSvSPLQ/export?format=csv&gid=1254825668'
    url_data2 = 'https://docs.google.com/spreadsheets/d/1-l1qJJpSdxjSwAj4AR9Nb0hd7ZV6FCpAtsGEgSvSPLQ/export?format=csv&gid=1906749374'

    print("Reading data...")
    try:
        # Load Data 1 (General)
        df1 = pd.read_csv(url_data1)
        df1 = df1.fillna('') # Cleaning NaN
        print(f"Data 1 (General) successfully read: {len(df1)} row found.")

        # Load Data 2 (Spesific)
        df2 = pd.read_csv(url_data2)
        df2.columns = df2.columns.str.strip() # Cleaning column names
        df2 = df2.fillna('') # Cleaning NaN
        print(f"Data 2 (Specific) successfully read: {len(df2)} product found.")

    except Exception as e:
        print(f"FAILED to read data from URL. Make sure the link is ‘public’. Error: {e}")
        exit()

    # 2. The second process DataFrame
    print("\nProcessing Data 1 (General)...")
    df_chunks_1 = process_data_1(df1)
    print(f"-> {len(df_chunks_1)} chunk general successfully created.")

    print("Processing Data 2 (Specific)...")
    df_chunks_2 = process_data_2(df2)
    print(f"-> {len(df_chunks_2)} chunk specific successfully created.")

    # 3. COMBINE BOTH
    df_combined = pd.concat([df_chunks_1, df_chunks_2], ignore_index=True)

    print("\n--- PROCESS COMPLETED ---")
    print(f"✅ A total of {len(df_combined)} chunks were successfully merged.")

    # 4. Save the final result
    file_output = "knowledge_base_FINAL_COMBINED.csv"
    df_combined.to_csv(file_output, index=False)

    print(f"✅ The final knowledge base (combined) has been successfully saved to: {file_output}")

    print("\n--- Example of the First 5 Lines of Combined Results ---")
    print(df_combined.head())

    print("\n--- Example of the Last 5 Lines of Combined Results  ---")
    print(df_combined.tail())

Reading data...
Data 1 (General) successfully read: 11 row found.
Data 2 (Specific) successfully read: 18 product found.

Processing Data 1 (General)...
-> 11 chunk general successfully created.
Processing Data 2 (Specific)...
-> 170 chunk specific successfully created.

--- PROCESS COMPLETED ---
✅ A total of 181 chunks were successfully merged.
✅ The final knowledge base (combined) has been successfully saved to: knowledge_base_FINAL_COMBINED.csv

--- Example of the First 5 Lines of Combined Results ---
        chunk_id                 main_topic  \
0   aqt-overview        Air Quality Testers   
1   wqt-overview      Water Quality Testers   
2   sqt-overview       Soil Quality Testers   
3   efs-overview  Educational Field Studies   
4  slac-overview  Science Labs and Campuses   

                                    chunk_title  \
0        Air Quality Testers (General Overview)   
1      Water Quality Testers (General Overview)   
2       Soil Quality Testers (General Overview)   
3  

In [ ]:
iseng = pd.read_csv("/content/knowledge_base_FINAL_COMBINED.csv")
iseng

,chunk_id,main_topic,chunk_title,chunk_text,src_url
0,aqt-overview,Air Quality Testers,Air Quality Testers (General Overview),Air quality testers play a vital role in monit...,https://enviroeducationtools.com/
1,wqt-overview,Water Quality Testers,Water Quality Testers (General Overview),Water quality testers are fundamental tools fo...,https://enviroeducationtools.com/
2,sqt-overview,Soil Quality Testers,Soil Quality Testers (General Overview),"Soil testers, such as moisture sensors, pH met...",https://enviroeducationtools.com/
3,efs-overview,Educational Field Studies,Educational Field Studies (General Overview),Field-based education becomes exponentially mo...,https://enviroeducationtools.com/
4,slac-overview,Science Labs and Campuses,Science Labs and Campuses (General Overview),"In science labs and campus facilities, environ...",https://enviroeducationtools.com/
...,...,...,...,...,...
176,tahdl-4,Temperature and Humidity Data Loggers,Key Features of Temperature and Humidity Data ...,Real-time Monitoring: Instantly view temperatu...,https://enviroeducationtools.com/soil-and-envi...
177,tahdl-5,Temperature and Humidity Data Loggers,Integrations for Temperature and Humidity Data...,Compatible with leading environmental software...,https://enviroeducationtools.com/soil-and-envi...
178,tahdl-6,Temperature and Humidity Data Loggers,Applications for Temperature and Humidity Data...,Science Education: Helps students learn about ...,https://enviroeducationtools.com/soil-and-envi...
179,tahdl-7,Temperature and Humidity Data Loggers,Industry Standards (Temperature and Humidity D...,ASTM E2877-13\nANSI/ASHRAE Standard 55-2020\nC...,https://enviroeducationtools.com/soil-and-envi...


In [ ]:
filtered = iseng[iseng['main_topic'] == 'Carbon Dioxide Meters']
filtered


,chunk_id,main_topic,chunk_title,chunk_text,src_url
22,cdm-1,Carbon Dioxide Meters,Carbon Dioxide Meters (General Description),Air Quality & Atmospheric Carbon Dioxide Meter...,https://enviroeducationtools.com/air-quality-a...
23,cdm-2,Carbon Dioxide Meters,Hardware for Carbon Dioxide Meters,High-accuracy CO₂ sensors (NDIR-based): Utiliz...,https://enviroeducationtools.com/air-quality-a...
24,cdm-3,Carbon Dioxide Meters,Software and Cloud Services for Carbon Dioxide...,Intuitive user interface with data logging\n\n...,https://enviroeducationtools.com/air-quality-a...
25,cdm-4,Carbon Dioxide Meters,Key Features of Carbon Dioxide Meters,Accurate CO₂ level detection in parts per mill...,https://enviroeducationtools.com/air-quality-a...
26,cdm-5,Carbon Dioxide Meters,Integrations for Carbon Dioxide Meters,Compatible with most standard HVAC systems \n\...,https://enviroeducationtools.com/air-quality-a...
27,cdm-6,Carbon Dioxide Meters,Benefits for Educators & Students (Carbon Diox...,Promotes awareness of indoor air quality among...,https://enviroeducationtools.com/air-quality-a...
28,cdm-7,Carbon Dioxide Meters,Applications for Carbon Dioxide Meters,Classroom air quality monitoring \n\nEnvironme...,https://enviroeducationtools.com/air-quality-a...
29,cdm-8,Carbon Dioxide Meters,Industry Standards (Carbon Dioxide Meters),ASHRAE Standard 62.1 \n\nEPA Indoor Air Qualit...,https://enviroeducationtools.com/air-quality-a...
30,cdm-9,Carbon Dioxide Meters,"Case Study: U.S. – Philadelphia, PA A...","U.S. – Philadelphia, PA\nA public school distr...",https://enviroeducationtools.com/air-quality-a...
31,cdm-10,Carbon Dioxide Meters,"Case Study: U.S. – Seattle, WA A...","U.S. – Seattle, WA\nA charter school integrate...",https://enviroeducationtools.com/air-quality-a...


# Embedding

In [ ]:
# --- SEL 2: IMPORTS & SETUP MODEL EMBEDDING (Free Version) ---


# No API Key required!

# Initialize SentenceTransformer embedding function
# This is your free “Magic Dictionary”.
# ChromaDB will automatically download 'all-MiniLM-L6-v2'
model_name = "all-MiniLM-L6-v2"
ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

print(f"✅ The library has been successfully imported and the ‘{model_name}’ embedding model is ready.")

✅ The library has been successfully imported and the ‘all-MiniLM-L6-v2’ embedding model is ready.


In [ ]:
# --- SEL 3: MUAT DATA CSV ---
file_path = "knowledge_base_FINAL_COMBINED.csv"

try:
    df = pd.read_csv(file_path)
    df = df.fillna('') # Ganti sel kosong (NaN) dengan string kosong
    print(f"✅ Berhasil memuat {len(df)} chunk dari {file_path}")

    # Tampilkan 5 baris pertama untuk memastikan data benar
    print("\nContoh data:")
    print(df.head())

except FileNotFoundError:
    print(f"Error: File {file_path} tidak ditemukan.")
    print("Pastikan Anda sudah meng-upload file ke panel 'Files' (ikon folder) di Colab 👈")
    raise SystemExit("File tidak ditemukan.")

✅ Berhasil memuat 181 chunk dari knowledge_base_FINAL_COMBINED.csv

Contoh data:
        chunk_id                 main_topic  \
0   aqt-overview        Air Quality Testers   
1   wqt-overview      Water Quality Testers   
2   sqt-overview       Soil Quality Testers   
3   efs-overview  Educational Field Studies   
4  slac-overview  Science Labs and Campuses   

                                    chunk_title  \
0        Air Quality Testers (General Overview)   
1      Water Quality Testers (General Overview)   
2       Soil Quality Testers (General Overview)   
3  Educational Field Studies (General Overview)   
4  Science Labs and Campuses (General Overview)   

                                          chunk_text  \
0  Air quality testers play a vital role in monit...   
1  Water quality testers are fundamental tools fo...   
2  Soil testers, such as moisture sensors, pH met...   
3  Field-based education becomes exponentially mo...   
4  In science labs and campus facilities, environ

In [ ]:
# --- SEL 4: INISIALISASI DATABASE ---
db_client = chromadb.PersistentClient(path="./my_chroma_db")
collection_name = "website_knowledge_free_model"

# Hapus collection lama jika ada (agar tidak bentrok model)
try:
    db_client.delete_collection(name=collection_name)
except Exception as e:
    pass # Tidak apa-apa jika belum ada

collection = db_client.get_or_create_collection(
    name=collection_name,
    embedding_function=ef # <-- PERUBAHAN UTAMA DI SINI
)

print(f"✅ Collection '{collection_name}' siap digunakan dengan model '{model_name}'.")

✅ Collection 'website_knowledge_free_model' siap digunakan dengan model 'all-MiniLM-L6-v2'.


In [ ]:
# --- SEL 5: TAMBAHKAN DATA KE DATABASE (STEP 3 & 4) ---
# INI ADALAH LANGKAH YANG HILANG!

# Siapkan data dari DataFrame 'df' (yang sudah di-load di Sel 3)
documents = df['chunk_text'].tolist()
metadatas = df[['main_topic', 'chunk_title', 'src_url']].to_dict('records')
ids = df['chunk_id'].tolist() # Menggunakan chunk_id unik Anda

try:
    print(f"Menambahkan {len(documents)} dokumen ke ChromaDB...")
    # Ini akan menggunakan model 'all-MiniLM-L6-v2' Anda secara lokal (gratis)
    print(f"Model '{model_name}' akan memproses teks secara lokal...")

    # Menambahkan data dalam batch (kelompok) agar lebih efisien
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        print(f"  Menambahkan batch {i//batch_size + 1} dari {len(documents)//batch_size + 1}...")

        batch_docs = documents[i:i+batch_size]
        batch_meta = metadatas[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]

        # Cek jika ID sudah ada sebelum menambahkan
        # Ini untuk mencegah error jika Anda menjalankan sel ini dua kali
        existing_ids = collection.get(ids=batch_ids)['ids']
        new_ids_to_add = [id for id in batch_ids if id not in existing_ids]

        # Jika semua ID sudah ada, lewati batch ini
        if not new_ids_to_add:
            print(f"    Batch {i//batch_size + 1} sudah ada di database, dilewati.")
            continue

        # Jika ada ID baru, kita perlu filter data untuk ID tersebut
        indices_to_add = [idx for idx, id in enumerate(batch_ids) if id in new_ids_to_add]

        final_batch_docs = [batch_docs[idx] for idx in indices_to_add]
        final_batch_meta = [batch_meta[idx] for idx in indices_to_add]
        final_batch_ids = [batch_ids[idx] for idx in indices_to_add]

        collection.add(
            documents=final_batch_docs,
            metadatas=final_batch_meta,
            ids=final_batch_ids
        )

    print(f"✅ Selesai! Sekarang ada {collection.count()} dokumen di database.")

except Exception as e:
    print(f"Terjadi error saat menambahkan data ke ChromaDB: {e}")

Menambahkan 181 dokumen ke ChromaDB...
Model 'all-MiniLM-L6-v2' akan memproses teks secara lokal...
  Menambahkan batch 1 dari 2...
  Menambahkan batch 2 dari 2...
✅ Selesai! Sekarang ada 181 dokumen di database.


In [ ]:
# Kode ini akan mengunduh file knowledge_base_FINAL_COMBINED.csv
# Anda bisa mengunduhnya langsung dari panel file di sebelah kiri
# atau menjalankan sel ini.
from google.colab import files

files.download('knowledge_base_FINAL_COMBINED.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- SEL 6: QUERY DATABASE (STEP 5) ---

print("\n--- Memulai Tes Query ---")

# GANTI PERTANYAAN INI UNTUK MENGETES
# user_question = "Apakah alat Anda bisa mendeteksi VOC di lab?"
# user_question = "What case studies do you have from Canada?"
# user_question = "Does it work with Google Sheets?"
user_question = "Tell me about the soil quality testers for systems?"

print(f"Pertanyaan Pengguna: \"{user_question}\"")

# Meng-query database
results = collection.query(
    query_texts=[user_question],
    n_results=3 # Ambil 3 hasil paling relevan
)

print("\n--- Hasil Pencarian (Konteks yang Paling Relevan) ---")

# Menampilkan hasil pencarian
if not results['documents'][0]:
    print("Tidak ada hasil yang ditemukan.")
else:
    for i, (doc, meta, dist) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        print(f"\nHasil #{i+1} (Jarak: {dist:.4f}):")
        print(f"  Judul Chunk: {meta['chunk_title']}")
        print(f"  Teks Chunk: \"{doc[:250]}...\"") # Tampilkan 250 karakter pertama
        print(f"  Sumber URL: {meta['src_url']}")

print("\n---")
print("Konteks di atas (results['documents']) adalah yang akan Anda berikan ke LLM (GPT) untuk dijawab.")


--- Memulai Tes Query ---
Pertanyaan Pengguna: "Tell me about the soil quality testers for systems?"

--- Hasil Pencarian (Konteks yang Paling Relevan) ---

Hasil #1 (Jarak: 0.3794):
  Judul Chunk: Industry Standards (Soil pH Testers)
  Teks Chunk: "EPA Soil Quality Guidelines
ASTM D4972 – Standard Test Method for pH of Soils
NRCS Soil Testing Recommendations (USDA)
Canadian Council of Ministers of the Environment (CCME) Soil Standards..."
  Sumber URL: https://enviroeducationtools.com/soil-and-environmental-health-testers/

Hasil #2 (Jarak: 0.4479):
  Judul Chunk: Soil Quality Testers (General Overview)
  Teks Chunk: "Soil testers, such as moisture sensors, pH meters, and nutrient analyzers, are transformative tools that significantly enhance curriculum delivery in agricultural science, biology, and environmental studies. They serve as the critical bridge between ..."
  Sumber URL: https://enviroeducationtools.com/

Hasil #3 (Jarak: 0.4501):
  Judul Chunk: Soil pH Testers (General De

In [ ]:
# --- SEL 7 (Baru): Instalasi untuk Hugging Face API ---
!pip install huggingface_hub

In [ ]:
# --- SEL 8 (New): RAG Loop with FREE API (CPU ONLY) - FIXED VERSION ---
from huggingface_hub import InferenceClient
from google.colab import userdata

# 1. Get your Hugging Face API Key
try:
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token:
        raise ValueError("HF Token not found")
except Exception as e:
    print("Error: Failed to retrieve HF_TOKEN from Colab Secrets.")
    print("Make sure you have saved it in the 🔑 Secrets Manager.")
    raise SystemExit(f"Error: {e}")

# 2. Initialize API Client
client = InferenceClient(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token
)

# 3. User Question
user_question = "can you give me the product software for air quality testers?"

print(f"Searching for context for: \"{user_question}\"")

# 4. Search in ChromaDB (This runs on CPU, 100% free)
# Assuming 'collection' is already defined and loaded
results = collection.query(
    query_texts=[user_question],
    n_results=3 # Get 3 contexts
)

# 5. Prepare Context
context_chunks = results['documents'][0]
if not context_chunks:
    context_str = "No relevant context found."
else:
    context_str = "\n\n".join(context_chunks)

# -------------------------------------------------------------------
# --- STEP 6 (MODIFICATION): Create Prompt (RAG) - "Chat" Format ---
# -------------------------------------------------------------------
# We are no longer creating one long string, but a list of 'messages'

system_prompt = """You are the ' Enviro Education Tools Product Selector & System Designer'. Your answers are for technical professionals who use American English.

Your task is to answer the user's question *strictly* and *only* based on the context provided. The context provided IS the information from https://enviroeducationtools.com/.
- Do not use any products, information, or technologies from other websites.
- Do not mention any other websites.
- Do not suggest prices or pricing information.

If the user asks 'about Enviro Education Tools' or 'where are you based', use the following information:
' Enviro Education Tools, based in Atlanta, Georgia, United States, is recognized as one of the leading global suppliers of advanced B2B and B2G (to a lesser degree, B2B2C, B2D) technologies in the world. For four decades,  Enviro Education Tools has served its customers in the U.S. and Canada, including many Fortune 500 companies, leading R&D firms, prestigious universities, and U.S. and Canadian government agencies. Asset Track Pro has invested heavily in R&D of its products and systems, has stringent quality assurance processes, and provides top-notch expert support remotely or onsite.'

If the answer is not in the context, say: 'I'm sorry, I do not have specific information on that topic from https://enviroeducationtools.com/'
"""

# Combine context and question for the "user"
user_content = f"""
Context:
{context_str}

Question:
{user_question}
"""

# This is the correct format for 'chat_completion'
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_content}
]

print("Context found, creating 'chat' prompt for API...")

# -------------------------------------------------------------------
# --- STEP 7 (MODIFICATION): Call Hugging Face API ---
# -------------------------------------------------------------------
print("Generating answer (calling HF Inference API)...")
response_text = ""
try:
    # Call API using 'chat_completion'
    # NOT 'text_generation'
    response = client.chat_completion(
        messages=messages,      # <-- Using 'messages' (not 'prompt')
        max_tokens=250,         # <-- 'max_new_tokens' changed to 'max_tokens'
        temperature=0.01        # <-- Turn off 'randomness' (replacement for do_sample=False)
    )

    # Get the answer from the 'chat_completion' response structure
    response_text = response.choices[0].message.content.strip()

except Exception as e:
    print(f"Error when calling API: {e}")
    response_text = "Sorry, an error occurred while contacting the AI model."

# Display the clean answer
print("\n--- CHATBOT ANSWER ---")
print(response_text)

Searching for context for: "can you give me the product software for air quality testers?"
Context found, creating 'chat' prompt for API...
Generating answer (calling HF Inference API)...

--- CHATBOT ANSWER ---
Based on the provided context, the product software for air quality testers is not explicitly mentioned. However, the features of the air quality testers are described as follows:

- Real-time monitoring of PM1.0, PM2.5, and PM10
- Plug-and-play operation with minimal calibration
- Color-coded air quality indicators for easy understanding
- Long-term data logging and remote access
- Compatible with local or school-wide displays
- Threshold-based alerts for poor air quality days

I'm sorry, I do not have specific information on the product software for air quality testers from https://enviroeducationtools.com/.


In [ ]:
# --- SEL 8 (New): RAG Loop with FREE API (CPU ONLY) - FIXED VERSION ---
from huggingface_hub import InferenceClient
from google.colab import userdata

# 1. Get your Hugging Face API Key
try:
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token:
        raise ValueError("HF Token not found")
except Exception as e:
    print("Error: Failed to retrieve HF_TOKEN from Colab Secrets.")
    print("Make sure you have saved it in the 🔑 Secrets Manager.")
    raise SystemExit(f"Error: {e}")

# 2. Initialize API Client
client = InferenceClient(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token
)

# 3. User Question
user_question = "can you give me the product software for air quality testers?"

print(f"Searching for context for: \"{user_question}\"")

# 4. Search in ChromaDB (This runs on CPU, 100% free)
# Assuming 'collection' is already defined and loaded
results = collection.query(
    query_texts=[user_question],
    n_results=3 # Get 3 contexts
)

# 5. Prepare Context
context_chunks = results['documents'][0]
if not context_chunks:
    context_str = "No relevant context found."
else:
    context_str = "\n\n".join(context_chunks)

# -------------------------------------------------------------------
# --- STEP 6 (MODIFICATION): Create Prompt (RAG) - "Chat" Format ---
# -------------------------------------------------------------------
# We are no longer creating one long string, but a list of 'messages'

system_prompt = """You are the ' Enviro Education Tools Product Selector & System Designer'. Your answers are for technical professionals who use American English.

Your task is to answer the user's question *strictly* and *only* based on the context provided. The context provided IS the information from https://enviroeducationtools.com/.
- Do not use any products, information, or technologies from other websites.
- Do not mention any other websites.
- Do not suggest prices or pricing information.

If the user asks 'about Enviro Education Tools' or 'where are you based', use the following information:
' Enviro Education Tools, based in Atlanta, Georgia, United States, is recognized as one of the leading global suppliers of advanced B2B and B2G (to a lesser degree, B2B2C, B2D) technologies in the world. For four decades,  Enviro Education Tools has served its customers in the U.S. and Canada, including many Fortune 500 companies, leading R&D firms, prestigious universities, and U.S. and Canadian government agencies. Asset Track Pro has invested heavily in R&D of its products and systems, has stringent quality assurance processes, and provides top-notch expert support remotely or onsite.'

If the answer is not in the context, say: 'I'm sorry, I do not have specific information on that topic from https://enviroeducationtools.com/'
"""

# Combine context and question for the "user"
user_content = f"""
Context:
{context_str}

Question:
{user_question}
"""

# This is the correct format for 'chat_completion'
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_content}
]

print("Context found, creating 'chat' prompt for API...")

# -------------------------------------------------------------------
# --- STEP 7 (MODIFICATION): Call Hugging Face API ---
# -------------------------------------------------------------------
print("Generating answer (calling HF Inference API)...")
response_text = ""
try:
    # Call API using 'chat_completion'
    # NOT 'text_generation'
    response = client.chat_completion(
        messages=messages,      # <-- Using 'messages' (not 'prompt')
        max_tokens=250,         # <-- 'max_new_tokens' changed to 'max_tokens'
        temperature=0.01        # <-- Turn off 'randomness' (replacement for do_sample=False)
    )

    # Get the answer from the 'chat_completion' response structure
    response_text = response.choices[0].message.content.strip()

except Exception as e:
    print(f"Error when calling API: {e}")
    response_text = "Sorry, an error occurred while contacting the AI model."

# Display the clean answer
print("\n--- CHATBOT ANSWER ---")
print(response_text)

Searching for context for: "can you give me the product software for air quality testers?"
Context found, creating 'chat' prompt for API...
Generating answer (calling HF Inference API)...

--- CHATBOT ANSWER ---
The product software for air quality testers is not explicitly mentioned in the provided context. However, the features of the air quality testers are described as follows:

- Real-time monitoring of PM1.0, PM2.5, and PM10
- Plug-and-play operation with minimal calibration
- Color-coded air quality indicators for easy understanding
- Long-term data logging and remote access
- Compatible with local or school-wide displays
- Threshold-based alerts for poor air quality days

These features suggest that the air quality testers are part of a comprehensive system that includes software for data logging, remote access, and display integration. However, the specific product software name is not mentioned in the provided context.


In [ ]:
# Kompres folder ChromaDB menjadi file zip
!zip -r my_chroma_db.zip ./my_chroma_db

print("✅ Folder 'my_chroma_db' berhasil dikompres menjadi 'my_chroma_db.zip'.")

updating: my_chroma_db/ (stored 0%)
updating: my_chroma_db/chroma.sqlite3 (deflated 49%)
updating: my_chroma_db/23479564-e50c-44ae-9517-ad1e7ef8a305/ (stored 0%)
updating: my_chroma_db/23479564-e50c-44ae-9517-ad1e7ef8a305/data_level0.bin (deflated 100%)
updating: my_chroma_db/23479564-e50c-44ae-9517-ad1e7ef8a305/link_lists.bin (stored 0%)
updating: my_chroma_db/23479564-e50c-44ae-9517-ad1e7ef8a305/header.bin (deflated 63%)
updating: my_chroma_db/23479564-e50c-44ae-9517-ad1e7ef8a305/length.bin (deflated 65%)
updating: my_chroma_db/2ca854c5-d396-4f5b-aa28-09dc6291a821/ (stored 0%)
updating: my_chroma_db/2ca854c5-d396-4f5b-aa28-09dc6291a821/data_level0.bin (deflated 100%)
updating: my_chroma_db/2ca854c5-d396-4f5b-aa28-09dc6291a821/link_lists.bin (stored 0%)
updating: my_chroma_db/2ca854c5-d396-4f5b-aa28-09dc6291a821/header.bin (deflated 63%)
updating: my_chroma_db/2ca854c5-d396-4f5b-aa28-09dc6291a821/length.bin (deflated 52%)
updating: my_chroma_db/d42ee4b7-d38f-45ff-a84f-7b595f5d371b/ (s